In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw02B.ipynb")

# Homework 2B: Food Safety (Continued)

## Cleaning and Exploring Data with `pandas`

## Due Date: Thursday, Sep 14, 11:59 PM
You must submit this assignment to Gradescope by the on-time deadline, Thursday, Sep 14, 11:59 PM. Please read the syllabus for the grace period policy. No late submissions beyond the grace period will be accepted. **We strongly encourage you to plan to submit your work to Gradescope several hours before the stated deadline.** This way, you will have ample time to reach out to staff for support if you encounter difficulties with submission. While course staff is happy to help guide you with submitting your assignment ahead of the deadline, we will not respond to last-minute requests for assistance (TAs need to sleep, after all!).

Please read the instructions carefully to submit your work to both the coding and written portals of Gradescope.

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** at the top of your notebook.

**Collaborators**: *list collaborators here*


## This Assignment

In this homework, we will continue our exploration of restaurant food safety scores for restaurants in San Francisco. The main goal for this assignment is to focus more on the analysis of the dataset, building on the data cleaning we have done earlier in HW 2A. 


After this homework, you should be comfortable with:
* Reading `pandas` documentation and using `pandas` methods,
* Working with data at different levels of granularity,
* Using `groupby` with different aggregation functions,
* Chaining different `pandas` functions and methods to find answers to exploratory questions.


## Score Breakdown 
Question | Manual | Points
--- | --- | ---
1a | no | 2
1b | no | 3
1c | no | 3
2a | no | 2
2b | no | 3
2c | no | 1
2d | no | 1
3a | no | 1
3b | no | 2
3c | yes | 3
3d | no | 3
3e | yes | 3
4 | no | 3
Total | 2 | 30


## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question. 

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run code, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.

Finally, unless we state otherwise, **do not use for loops or list comprehensions**. The majority of this assignment can be done using built-in commands in `pandas` and `NumPy`.  Our autograder isn't smart enough to check, but you're depriving yourself of key learning objectives if you write loops / comprehensions, and you also won't be ready for the midterm.


In [2]:
import numpy as np
import pandas as pd

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

In HW 2A, we took you through the entire process of reading data from a file to perform some exploration of the data. Here, we again load the dataset that we will be using in HW 2B along with some of the columns we had added in HW 2A. For any additional context regarding the dataset, feel free to revisit HW 2A.

In [3]:
bus = pd.read_csv('data/bus.csv', encoding='ISO-8859-1').rename(columns={"business id column": "bid"})
bus['postal5'] = bus['postal_code'].str[:5]
ins = pd.read_csv('data/ins.csv')
ins['timestamp'] = pd.to_datetime(ins['date'], format='%m/%d/%Y %I:%M:%S %p')
ins['bid'] = ins['iid'].str.split("_", expand=True)[0].astype(int) 

# This code is essential for the autograder to function properly. Do not edit.
ins_test = ins

<br/><br/>

---


# Question 1: Inspecting the Inspections


## Question 1a

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [4]:
ins.head(5)

,iid,date,score,type,timestamp,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,2019-03-29,100010
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,2019-04-17,100017
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,2019-08-26,100017


To better understand how the scores have been allocated, examine how the maximum score varies for each type of inspection. Create a `DataFrame` object `ins_score_by_type`, indexed by all the inspection types (e.g., New Construction, Routine - Unscheduled, etc.), with a single column named `max_score` containing the highest score received. You may find `pd.rename()` to be useful!

In [5]:
ins["score"] = ins["score"].astype('int64')
ins_score_by_type = ins[["type", "score"]].groupby("type").max().rename({"score": "max_score"}, axis='columns')
ins_score_by_type

,max_score
type,
Administrative or Document Review,-1
Community Health Assessment,-1
Complaint,-1
Complaint Reinspection/Followup,-1
Foodborne Illness Investigation,-1
Multi-agency Investigation,-1
New Construction,-1
New Ownership,-1
New Ownership - Followup,-1


In [6]:
grader.check("q1a")

q1a results: All test cases passed!

<br/>

---

## Question 1b


Given the variability of `ins['score']` observed in 1.a, let's examine the inspection scores `ins['score']` further.

In [6]:
ins['score'].value_counts().head()

score
-1      12632
 100     1993
 96      1681
 92      1260
 94      1250
Name: count, dtype: int64

There are a large number of inspections with a `score` of `-1`. These are probably missing values. Let's see what types of inspections have scores and which do not (score of -1).  We have defined for you a new column `'Missing Score'` that shows `True` if the score for that business is `-1` to help you out with the analysis. 

Use `.groupby` to find out the number of scores that every combination of `type` and `Missing Score` can take on. The result should be a **`DataFrame`** that should look **exactly** as shown below:

<center> <img src="pics/1b.png" width="400"/> 


In [7]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")
ins_missing_score_group = ins.groupby(["type", "Missing Score"]) \
    .agg(lambda x: len(x))[["iid"]] \
    .rename({"iid":"Count"}, axis="columns")
ins_missing_score_group

,,Count
type,Missing Score,
Administrative or Document Review,True,4
Community Health Assessment,True,1
Complaint,True,1458
Complaint Reinspection/Followup,True,227
Foodborne Illness Investigation,True,115
Multi-agency Investigation,True,3
New Construction,True,994
New Ownership,True,1592
New Ownership - Followup,True,499


In [8]:
grader.check("q1b")

q1b results: All test cases passed!

<br/>

---

## Question 1c


Using `groupby` to perform the above analysis gave us a `DataFrame` that wasn't the most readable at first glance. There are better ways to represent the above information that take advantage of the fact that we are looking at combinations of two variables. It's time to pivot (pun intended)!

Create the following `DataFrame`, and assign it to to the variable `ins_missing_score_pivot`. You'll want to use the `pivot_table` method of the `DataFrame` class, which you can read about in the pivot_table [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html). Once you create `ins_missing_score_pivot`, add another column titled `'Total'`, which contains the total number of inspections of that `type`. Sort the table by descending order of `'Total'`.

**Hint:** Consider what happens if there are no values that correspond to a particular combination of `'Missing Score'` and `'type'`. Looking at the documentation for `pivot_table`, is there any function argument that allows you to specify what value to fill in?

If you've done everything right, you should observe that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections and that `ins_missing_score_pivot` looks exactly like below:


<table border="1" class="dataframe" >  <thead>    
    <tr style="text-align: right;">      <th>Missing Score</th>      <th>False</th>      <th>True</th>      <th>Total</th>    </tr>    <tr align="right">      <th>type</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    
    <tr  align="right">      <th>Routine - Unscheduled</th>      <td>14031</td>      <td>46</td>      <td>14077</td>    </tr>    
    <tr  align="right">      <th>Reinspection/Followup</th>      <td>0</td>      <td>6439</td>      <td>6439</td>    </tr>    
    <tr  align="right">      <th>New Ownership</th>      <td>0</td>      <td>1592</td>      <td>1592</td>    </tr>    
    <tr  align="right">      <th>Complaint</th>      <td>0</td>      <td>1458</td>      <td>1458</td>    </tr>    
    <tr  align="right">      <th>New Construction</th>      <td>0</td>      <td>994</td>      <td>994</td>    </tr>    
    <tr  align="right">      <th>Non-inspection site visit</th>      <td>0</td>      <td>811</td>      <td>811</td>    </tr>    
    <tr  align="right">      <th>New Ownership - Followup</th>      <td>0</td>      <td>499</td>      <td>499</td>    </tr>    
    <tr  align="right">      <th>Structural Inspection</th>      <td>0</td>      <td>394</td>      <td>394</td>    </tr>    
    <tr  align="right">      <th>Complaint Reinspection/Followup</th>      <td>0</td>      <td>227</td>      <td>227</td>    </tr>    
    <tr  align="right">      <th>Foodborne Illness Investigation</th>      <td>0</td>      <td>115</td>      <td>115</td>    </tr>    
    <tr  align="right">      <th>Routine - Scheduled</th>      <td>0</td>      <td>46</td>      <td>46</td>    </tr>    
    <tr  align="right">      <th>Administrative or Document Review</th>      <td>0</td>      <td>4</td>      <td>4</td>    </tr>    
    <tr  align="right">      <th>Multi-agency Investigation</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Special Event</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Community Health Assessment</th>      <td>0</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>


In [19]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")
ins_missing_score_pivot = ins.pivot_table(index="type",
                                         columns="Missing Score",
                                         values="iid",
                                         margins=True,
                                         margins_name="Total",
                                         aggfunc=np.size,
                                         fill_value=0).sort_values(by="Total", ascending=False) 
ins_missing_score_pivot = ins_missing_score_pivot[ins_missing_score_pivot.index != "Total"]
ins_missing_score_pivot

Missing Score,False,True,Total
type,,,
Routine - Unscheduled,14031,46,14077
Reinspection/Followup,0,6439,6439
New Ownership,0,1592,1592
Complaint,0,1458,1458
New Construction,0,994,994
Non-inspection site visit,0,811,811
New Ownership - Followup,0,499,499
Structural Inspection,0,394,394
Complaint Reinspection/Followup,0,227,227


In [20]:
grader.check("q1c")

q1c results: All test cases passed!

Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. It is reasonable for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but we might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

<br/><br/>

---

# Question 2: Joining Data Across Tables

In this question, we will start to connect data across multiple tables. We will be using the `merge` function. 

<br/>

--- 

## Question 2a

Let's figure out which restaurants had the lowest scores. Before we proceed, let's filter out missing scores from `ins` so that negative scores don't influence our results. 

Note that there might be something interesting we could learn from businesses with missing scores, but we are omitting such analysis from this homework. You might consider exploring this for the optional question at the end. 

Note: We have no idea if there is actually anything interesting to learn as we have not attempted this ourselves.

In [21]:
ins = ins[ins["score"] > 0]

We'll start by creating a new `DataFrame` called `ins_named`. It should be exactly the same as `ins`, except that it should have the name and address of every business, as determined by the `bus` DataFrame. 

**Hint**: Use the `merge` method to join the `ins` DataFrame with the appropriate portion of the `bus` DataFrame. See the official [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) on how to use `merge`. The first few rows of the resulting `DataFrame` you create are shown below:

<img src="pics/2a.png" width="1080"/>

In [22]:
ins_named = pd.merge(left=ins, right=bus[["bid", "name", "address"]], left_on="bid", right_on="bid")
ins_named.head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address
0,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,2019-04-03,100010,False,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B
1,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,2019-08-16,100017,False,AMICI'S EAST COAST PIZZERIA,475 06th St
2,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,2019-05-20,100041,False,UNCLE LEE CAFE,3608 BALBOA ST
3,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,2019-04-25,100055,False,Twirl and Dip,335 Martin Luther King Jr. Dr
4,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,2019-09-12,100055,False,Twirl and Dip,335 Martin Luther King Jr. Dr


In [23]:
grader.check("q2a")

q2a results: All test cases passed!

<br/>

--- 

## Question 2b

Let's look at the 20 businesses in `ins_named` with the lowest **median** score. Order your results by the median score followed by the business name to break ties. The resulting table should look like the table below.

This one is pretty challenging! Don't forget to rename the `score` column. 

**Hint**: The `agg` function can accept a dictionary as an input. See the [agg documentation](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html). Additionally, when thinking about what aggregation functions to use, ask yourself what value would be in the `"name"` column for each entry across the group? Can we select just one of these values to represent the whole group?

As usual, **YOU SHOULD NOT USE LOOPS OR LIST COMPREHENSIONS**. Try and break down the problem piece by piece instead, gradually chaining together different `pandas` functions. Feel free to use more than one line!

<table border="1" class="dataframe">  <thead>    
    <tr style="text-align: right;">      <th></th>      <th>name</th>      <th>median score</th>    </tr> 
    <tr  align="right">  <th align="right">bid</th>      <th></th>      <th></th>    </tr> </thead>  <tbody>    
    <tr  align="right">      <th>84590</th>      <td>Chaat Corner</td>      <td>54.0</td>    </tr>    
    <tr  align="right">        <th>90622</th>      <td>Taqueria Lolita</td>      <td>57.0</td>    </tr>    
    <tr  align="right">         <th>94351</th>      <td>VBowls LLC</td>      <td>58.0</td>    </tr>    
    <tr  align="right">          <th>69282</th>      <td>New Jumbo Seafood Restaurant</td>      <td>60.5</td>    </tr>    
    <tr  align="right">         <th>1154</th>      <td>SUNFLOWER RESTAURANT</td>      <td>63.5</td>    </tr>  
    <tr  align="right">          <th>93150</th>      <td>Chez Beesen</td>      <td>64.0</td>    </tr>   
    <tr  align="right">     <th>39776</th>      <td>Duc Loi Supermarket</td>      <td>64.0</td>    </tr>  
    <tr  align="right">         <th>78328</th>      <td>Golden Wok</td>      <td>64.0</td>    </tr>  
    <tr  align="right">          <th>69397</th>      <td>Minna SF Group LLC</td>      <td>64.0</td>    </tr>     
    <tr  align="right">        <th>93502</th>      <td>Smoky Man</td>      <td>64.0</td>    </tr>    
    <tr  align="right">           <th>98995</th>      <td>Vallarta's Taco Bar</td>      <td>64.0</td>    </tr>    
    <tr  align="right">         <th>10877</th>      <td>CHINA FIRST INC.</td>      <td>64.5</td>    </tr>    
    <tr  align="right">        <th>71310</th>      <td>Golden King Vietnamese Restaurant</td>      <td>64.5</td>    </tr>     
    <tr  align="right">          <th>89070</th>      <td>Lafayette Coffee Shop</td>      <td>64.5</td>    </tr>
    <tr  align="right">          <th>71008</th>      <td>House of Pancakes</td>      <td>65.0</td>    </tr> 
    <tr  align="right">         <th>2542</th>      <td>PETER D'S RESTAURANT</td>      <td>65.0</td>    </tr>           
    <tr  align="right">        <th>3862</th>      <td>IMPERIAL GARDEN SEAFOOD RESTAURANT</td>      <td>66.0</td>    </tr>   
    <tr  align="right">         <th>61427</th>      <td>Nick's Foods</td>      <td>66.0</td>    </tr>    
    <tr  align="right">          <th>72176</th>      <td>Wolfes Lunch</td>      <td>66.0</td>    </tr>    
    <tr  align="right">        <th>89141</th>      <td>Cha Cha Cha on Mission</td>      <td>66.5</td>    </tr>  </tbody></table>

In [15]:
twenty_lowest_scoring = ins_named.groupby("bid")[["name", "score"]] \
    .agg({"name": lambda x: x.iloc[0], "score": lambda x: x.median()}) \
    .sort_values(by=["score", "name"]).head(20).rename({"score": "median score"}, axis="columns")

twenty_lowest_scoring

,name,median score
bid,,
84590,Chaat Corner,54.0
90622,Taqueria Lolita,57.0
94351,VBowls LLC,58.0
69282,New Jumbo Seafood Restaurant,60.5
1154,SUNFLOWER RESTAURANT,63.5
93150,Chez Beesen,64.0
39776,Duc Loi Supermarket,64.0
78328,Golden Wok,64.0
69397,Minna SF Group LLC,64.0


In [16]:
grader.check("q2b")

q2b results: All test cases passed!

<br/>

--- 
## Question 2c

Let's figure out which restaurant had the worst score ever (single lowest score). 

In the cell below, assign `worst_restaurant` to the name of the restaurant with the **lowest inspection score ever**. We should not be considering restaurants with missing scores, so this should not be a retaurant that has a score of `-1`. For fun: Look up the reviews for this restaurant on Yelp. Do you see any reviews that indicate this restaurant had health inspection issues?


In [17]:
worst_restaurant = ins_named.groupby("bid")[["name", "score"]] \
    .agg({"name": lambda x: x.iloc[0], "score": lambda x: x.min()}) \
    .sort_values(by=["score", "name"]).iloc[0]["name"]
worst_restaurant

'Lollipot'

In [18]:
grader.check("q2c")

q2c results: All test cases passed!

<br/>

--- 
## Question 2d

Did this restaurant clean up its act? Look in the database to see if it scored better on its next inspection. Assign `cleaned_up` to `True` or `False`, depending on whether it performed better or not.

In [19]:
# SCRATCH WORK

In [20]:
scores = ins_named[ins_named["name"] == worst_restaurant].sort_values(by="timestamp")
scores = scores["score"].reset_index()
min_idx = scores.idxmin()
cleaned_up = scores.iloc[min_idx["score"]]["score"] < scores.iloc[min_idx["score"] + 1]["score"]
cleaned_up

True

In [21]:
grader.check("q2d")

q2d results: All test cases passed!

<br/><br/>

---
# Question 3: I Scream, You Scream 

Now that you've analyzed and found out which restaurants to avoid in SF, we can turn toward the more interesting question of what dessert places are the best! For the purposes of this question, we assume that ice cream is the best dessert (and rightfully so!)

<br/>

--- 
## Question 3a

In your quest to find the best ice cream shop, the first step is to find all the businesses in `ins_named` that **contain the word 'ice cream'** in their `name`, and assign the resulting `DataFrame` to `ice_cream_shops`. To help you out, we created the `lowercase_name` column so you do not need to worry about checking for capitalized letters when checking if `name` contains 'ice cream'.

**Hint:** You might find the `.str` accessors useful yet again!

In [22]:
ins_named['lowercase_name'] = ins_named['name'].str.lower()
ice_cream_shops = ins_named[ins_named["lowercase_name"].str.contains("ice cream")]
ice_cream_shops.head(10)

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
152,102067_20190911,09/11/2019 12:00:00 AM,100,Routine - Unscheduled,2019-09-11,102067,False,Humphry Slocombe Ice Cream,1 Ferry Building Plaza,humphry slocombe ice cream
559,14743_20161103,11/03/2016 12:00:00 AM,100,Routine - Unscheduled,2016-11-03,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream
560,14743_20170928,09/28/2017 12:00:00 AM,98,Routine - Unscheduled,2017-09-28,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream
561,14743_20180716,07/16/2018 12:00:00 AM,100,Routine - Unscheduled,2018-07-16,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream
562,14743_20190920,09/20/2019 12:00:00 AM,96,Routine - Unscheduled,2019-09-20,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream
2158,31_20180615,06/15/2018 12:00:00 AM,96,Routine - Unscheduled,2018-06-15,31,False,Norman's Ice Cream and Freezes,2801 Leavenworth St,norman's ice cream and freezes
2159,31_20190702,07/02/2019 12:00:00 AM,94,Routine - Unscheduled,2019-07-02,31,False,Norman's Ice Cream and Freezes,2801 Leavenworth St,norman's ice cream and freezes
2267,32733_20161020,10/20/2016 12:00:00 AM,96,Routine - Unscheduled,2016-10-20,32733,False,Xanath Ice Cream,951 Valencia St,xanath ice cream
2268,32733_20170808,08/08/2017 12:00:00 AM,96,Routine - Unscheduled,2017-08-08,32733,False,Xanath Ice Cream,951 Valencia St,xanath ice cream
2269,32733_20180214,02/14/2018 12:00:00 AM,94,Routine - Unscheduled,2018-02-14,32733,False,Xanath Ice Cream,951 Valencia St,xanath ice cream


In [23]:
grader.check("q3a")

q3a results: All test cases passed!

<br/>

--- 
## Question 3b

Assign to `ice_cream_at_least_3` a `DataFrame` consisting of only those ice cream shops that have had at least 3 inspections. Remember, the `bid` uniquely defines an ice cream shop at a particular location, not its `name`!

In [24]:
ice_cream_at_least_3 = ice_cream_shops.groupby("bid").filter(lambda x: len(x) >= 3)
ice_cream_at_least_3.head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
559,14743_20161103,11/03/2016 12:00:00 AM,100,Routine - Unscheduled,2016-11-03,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream
560,14743_20170928,09/28/2017 12:00:00 AM,98,Routine - Unscheduled,2017-09-28,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream
561,14743_20180716,07/16/2018 12:00:00 AM,100,Routine - Unscheduled,2018-07-16,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream
562,14743_20190920,09/20/2019 12:00:00 AM,96,Routine - Unscheduled,2019-09-20,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream
2267,32733_20161020,10/20/2016 12:00:00 AM,96,Routine - Unscheduled,2016-10-20,32733,False,Xanath Ice Cream,951 Valencia St,xanath ice cream


In [25]:
grader.check("q3b")

q3b results: All test cases passed!

<!-- BEGIN QUESTION -->

<br/>

---
## Question 3c

In the cell below, run the following line of code: `q3c_df = ice_cream_shops.sort_values('timestamp').groupby('bid').agg('first')`

Is the granularity of `ice_cream_at_least_3` the same as the granularity of `q3c_df`? In other words, what does a single row of `q3c_df` represent, and what does a single row in ice_cream_at_least_3 represent? Explain the granularity of each `DataFrame`. Your answer does not need to be more than 2-3 lines, but you should be specific.

In [26]:
q3c_df = ice_cream_shops.sort_values('timestamp').groupby('bid').agg('first')
q3c_df.head()

,iid,date,score,type,timestamp,Missing Score,name,address,lowercase_name
bid,,,,,,,,,
31,31_20180615,06/15/2018 12:00:00 AM,96,Routine - Unscheduled,2018-06-15,False,Norman's Ice Cream and Freezes,2801 Leavenworth St,norman's ice cream and freezes
758,758_20171212,12/12/2017 12:00:00 AM,94,Routine - Unscheduled,2017-12-12,False,BAKERY/ICE CREAM/STOREROOM,2 New Montgomery St,bakery/ice cream/storeroom
4671,4671_20170117,01/17/2017 12:00:00 AM,98,Routine - Unscheduled,2017-01-17,False,MARCO POLO ITALIAN ICE CREAM,1447 TARAVAL St,marco polo italian ice cream
5032,5032_20170627,06/27/2017 12:00:00 AM,94,Routine - Unscheduled,2017-06-27,False,MITCHELLS ICE CREAM,688 SAN JOSE Ave,mitchells ice cream
5524,5524_20161011,10/11/2016 12:00:00 AM,98,Routine - Unscheduled,2016-10-11,False,AT&T Park - Coffee and Ice Cream (4A+4B),24 WILLIE MAYS PLAZA,at&t park - coffee and ice cream (4a+4b)


The granularity of `ice_cream_at_least_3` is different from `q3c_df`. A single row of `ice_cream_at_least_3` represents one of the reviews of a restaurant that has at least 3 reviews. A single row of `q3c_df` represents the earliest review of a restaurant. In other word, `q3c_df` aggregates multiple reviews into one. That's why `ice_cream_at_least_3` has higher granularity.

<!-- END QUESTION -->

<br/>

---
## Question 3d

Rather than the inspection scores, you find that the number of vowels present in the business `name` is a better indicator of how good the ice cream is when it comes to the shops in `ice_cream_at_least_3`. Using the helper function `count_vowels` we have defined for you, sort all the ice cream shops in `ice_cream_at_least_3` based on this metric in descending order. Then, return a **Python `list`** consisting of the top 2 **uniquely named** ice cream shops using this sorted `DataFrame`. You should break ties using alphabetical ordering. You do not need to stick to the skeleton code provided, but you are **not allowed to do not add any new columns!**

This is pretty challenging, but rest assured, the price of knowing the best ice cream shops is well worth it! 


In [27]:
def count_vowels(name):
    vowels = 'aeiouAEIOU'
    count = 0
    return sum([letter in vowels for letter in name])

In [28]:
sorted_by_vowel_count = ice_cream_at_least_3[["name"]].groupby("name").first().reset_index() \
                        .sort_values(by="name", key=lambda x: x.apply(count_vowels), ascending=False)
top_2_ice_cream = sorted_by_vowel_count.iloc[:2]["name"].tolist()
top_2_ice_cream

['MARCO POLO ITALIAN ICE CREAM', 'AT&T Park - Coffee and Ice Cream (4A+4B)']

In [29]:
grader.check("q3d")

q3d results: All test cases passed!

<!-- BEGIN QUESTION -->

<br/>

---
## Question 3e

Finally, to examine different parts of a chained pandas statement, describe the purpose of each of the functions used (`.loc`, `.groupby`, `idxmax()`) in words. 

Secondly, share what you think this line of code accomplishes. In other words, write a question that could be answered using this statement.

While the first part of this question will be graded for correctness, the second part of this question is a bit more open-ended. Answers demonstrating your understanding will get full credit.

In [30]:
ice_cream_at_least_3.loc[ice_cream_at_least_3.groupby("bid")["score"].idxmax()].head()

,iid,date,score,type,timestamp,bid,Missing Score,name,address,lowercase_name
3704,4671_20171130,11/30/2017 12:00:00 AM,100,Routine - Unscheduled,2017-11-30,4671,False,MARCO POLO ITALIAN ICE CREAM,1447 TARAVAL St,marco polo italian ice cream
3972,5032_20190718,07/18/2019 12:00:00 AM,100,Routine - Unscheduled,2019-07-18,5032,False,MITCHELLS ICE CREAM,688 SAN JOSE Ave,mitchells ice cream
4182,5524_20170919,09/19/2017 12:00:00 AM,100,Routine - Unscheduled,2017-09-19,5524,False,AT&T Park - Coffee and Ice Cream (4A+4B),24 WILLIE MAYS PLAZA,at&t park - coffee and ice cream (4a+4b)
4186,5528_20170424,04/24/2017 12:00:00 AM,100,Routine - Unscheduled,2017-04-24,5528,False,AT&T - Juma Cart 1 - Ice Cream,24 WILLIE MAYS PLAZA,at&t - juma cart 1 - ice cream
559,14743_20161103,11/03/2016 12:00:00 AM,100,Routine - Unscheduled,2016-11-03,14743,False,Polly Ann Ice Cream,3138 Noriega St.,polly ann ice cream


#### First Part

- `.loc`: locate the rows with the given indices.
- `.groupby`: group reviews for each restaurant respectively.
- `idxmax()`: obtain the index of the row that has the maximum score in a given group.

#### Second Part

- Among the restaurants that have at least 3 reviews, obtain the review with the highest score of each restaurant respectively.

In [31]:
# You may do some scratch work in this cell, however, only your written answer will be graded. 
# Any outputs or dataframes you generate here will not be counted as part of your explanation.

<!-- END QUESTION -->

<br/><br/>

---

# Question 4: Restaurant Ratings Over Time

As a final challenge, we consider a scenario involving restaurants with multiple ratings over time.

Let's see which restaurant location has had the most extreme improvement in its scores. Let the "swing" of a restaurant location be defined as the difference between its highest-ever and lowest-ever score. **Only consider restaurant locations with at least 3 scores—that is, restaurants that were rated at least 3 times.** Assign `max_swing` to the name of the restaurant that has the maximum swing. 

We have not provided any skeleton, as there are many paths to getting the correct answer. The recommended approach to solving this problem is to break it down into smaller chunks (e.g., first, ensure all restaurants have at least 3 scores; second, compute ... ). This will likely require more than one line, so feel free to add/remove columns and define new temporary variables. Remember to assign your solution - a string containing the `name` of the restaurant location that experienced the most extreme improvement - to `max_swing` after you do so. 

**Note**: The "swing" is of a specific restaurant location. There might be some restaurants with multiple locations; we are focusing on the swing of a particular restaurant as specified by its `name` and `address`.



In [32]:
restaurant_at_least_3 = ins_named.groupby(["name", "address"]).filter(lambda x: len(x) >= 3)
swings = restaurant_at_least_3[["name", "address","score"]].groupby(["name", "address"]) \
                    .agg(lambda x: x.max() - x.min()).rename({"score": "max_swing"}, axis="columns") \
                    .sort_values(by="max_swing", ascending=False)
max_swing = swings.iloc[0].name[0]
max_swing

'Lollipot'

In [33]:
grader.check("q4")

q4 results: All test cases passed!

## Summary of Inspections Data

We have done a lot in this homework! 
 
- Broke down the inspection scores in detail using `.groupby` and `.pivot_table`,
- Joined the business and inspection data and identified the name of the restaurant with the worst rating,
- Took a deep dive into the sweet world of ice cream and found the best spots under varying metrics,
- Took a swing at analyzing how restaurant inspection scores change over time!

Over the course of this 2-part homework, we hope you have become more familiar with `pandas` - in terms of identifying when to use particular functions, how they work, when they can support EDA - as well as about EDA and Data Cleaning, as part of the broader Data Science Lifecycle. These tools will serve you well as a data scientist!

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Congratulations! You have finished Homework 2B! ##

### Submission Instructions

Below, you will see two cells.
* Running the very last cell will automatically generate a zip with your autograded answers.
* Only after you run the very last cell, you can press button on the second last cell to automatically generate a PDF of all manually graded questions in Homework 2B. Clicking on the link should open up the pdf in Jupyer and allow you to download it from there. If the link does not work, please check `File Browser` tab on the left panel and download hw02B.pdf from there.

To receive credit on this assignment, **you must submit both your coding and written portions to their respective Gradescope portals.** Your written submission (a single
PDF) can be generated as follows:

1. Access your answers to manually graded Homework 2B  questions in one of three ways:
      * <b>*Automatically create PDF (recommended)*</b>: Run the first cell below and download the generated PDF. This function will extract your response to the manually graded questions and put them on a separate page. This process may fail if your answer is not properly formatted; if this is the case, check out common errors and solutions described on Ed or follow either of the two ways described below.
      * *Manually download PDF*: If there are issues with automatically generating the PDF in the first cell, you can try downloading the notebook as a PDF by clicking on `File -> Save and Export Notebook As... -> PDF`. If you choose to go this route, you must take special care to ensure all appropriate pages are chosen for each question on Gradescope.
      * *Take screenshots*: If that doesn't work either, you can take screenshots of your answers (and your code if present) to manually graded questions and include them as images in a PDF. The manually graded questions are listed at the top of the Homework 2B notebook.
    
1. **Important**: When submitting on Gradescope, you **must tag pages to each question correctly** (it prompts you to do this after submitting your work). This significantly streamlines the grading process for our readers. Failure to do this may result in a score of 0 for untagged questions.

**You are responsible for ensuring your submission follows our requirements. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

After you have run the cell below and generated the zip file, you can open the PDF <a href='hw02B.pdf' download>here</a>.

In [34]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)

Running your submission against local test cases...



Your submission received the following results when run against available test cases:

    q1a results: All test cases passed!

    q1b results: All test cases passed!

    q1c results: All test cases passed!

    q2a results: All test cases passed!

    q2b results: All test cases passed!

    q2c results: All test cases passed!

    q2d results: All test cases passed!

    q3a results: All test cases passed!

    q3b results: All test cases passed!

    q3d results: All test cases passed!

    q4 results: All test cases passed!
